# Introduzione a Redis

## Nozioni Base
Redis is an open source (BSD licensed), in-memory data structure store used as a database, cache, message broker, and streaming engine. Redis provides data structures such as strings, hashes, lists, sets, sorted sets with range queries, bitmaps, hyperloglogs, geospatial indexes, and streams. Redis has built-in replication, Lua scripting, LRU eviction, transactions, and different levels of on-disk persistence, and provides high availability via Redis Sentinel and automatic partitioning with Redis Cluster.

You can run atomic operations on these types, like appending to a string; incrementing the value in a hash; pushing an element to a list; computing set intersection, union and difference; or getting the member with highest ranking in a sorted set.

To achieve top performance, Redis works with an in-memory dataset. Depending on your use case, Redis can persist your data either by periodically dumping the dataset to disk or by appending each command to a disk-based log. You can also disable persistence if you just need a feature-rich, networked, in-memory cache.

Redis supports asynchronous replication, with fast non-blocking synchronization and auto-reconnection with partial resynchronization on net split.

Redis also includes:
- Transactions
- Pub/Sub
- Lua scripting
- Keys with a limited time-to-live
- LRU eviction of keys
- Automatic failover
You can use Redis from most programming languages.

Redis is written in ANSI C and works on most POSIX systems like Linux, *BSD, and Mac OS X, without external dependencies. Linux and OS X are the two operating systems where Redis is developed and tested the most, and we recommend using Linux for deployment. Redis may work in Solaris-derived systems like SmartOS, but support is best effort. There is no official support for Windows builds.

## Chi usa Redis?
Le compagnie più note che usano Redis sono:
- Twitter
- Github
- Snapchat
- Cragslist
- StackOverflow

E molte altre ancora.

## Installazione
### Windows
Redis non è supportato su Windows, ma si può installare sfruttando la WSL2 (Windows Subsystem for Linux).
Con una shell WSL correttamente inizializzata si potranno seguire gli step di installazione su macchina Ubuntu/Debian per installare l'ultima versione di Redis.

```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```

E per avviare il servizio basterà lanciare il comando:
```bash
sudo service redis-server start
```

### Linux
Avendo in commercio una gran quantità di distribuzioni Linux, ogni package manager di riferimento ha una guida su come installare il pacchetto Redis.
Come spiegato nel capitolo per Windows, su sistemi Debian/Ubuntu possiamo aggiungere il Repository APT Ufficiale `packages.redis.io` e installare successivamente il pacchetto:
```bash
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg

echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list

sudo apt-get update
sudo apt-get install redis
```

Alternativamente possiamo sfruttare il pacchetto Redis disponibile sullo Snapcraft Store e installarlo su tutte le distribuzioni che lo supportano:

```bash
sudo snap install redis
```

### MacOS
Per installare il pacchetto Redis su dispositivi macOS, sarà necessario in prima battuta installare Brew, un package manager creato per il Sistema Operativo di Cupertino.
Una volta installato correttamente si dovrà semplicemente lanciare il comando:
```bash
brew install redis
```

Per avviare il servizio si dovrà usare:
```bash
redis-server
```

Per fermare il servizio andrà premuto `CTRL-C`

## Pro e Contro

## Librerie del DBMS
Quasi tutti i linguaggi di programmazione hanno disponibile un porting per connettersi al servizio Redis, lavorando con Python andremo nel dettaglio di quest'ultima.
### redis-py
**Installazione**

Per installare redis-py:
```bash
pip install redis-py
```

Per migliorare le performance si può utilizzare installa il pacchetto con il supporto ad `hiredis`, che offre un Response Parser compilato, che per molte situazioni richiede 0 modifiche.

```bash
pip install redis[hiredis]
```

In [ ]:
import sys
!{sys.executable} -m pip install redis

## Creazione e Connessione al DB
Per facilitare lo sviluppo su più sistemi operativi, questo progetto dispone di un docker-compose.yml file, contenente il necessario per avviare un'istanza redis con due User Interfaces, quali `redis-cli` e `RedisInsight`.

Per avviare il servizio si può lanciare il comando:
```bash
make start
```

### Connessione al DB tramite redis-py

In [15]:
import redis

r = redis.Redis(host='localhost', port=6379, decode_responses=True)

Prove comandi

In [16]:
r.set('nome', 'Marco')

True

In [17]:
r.get('nome')

'Marco'

In [18]:
r.hset('user-session:123', mapping={
    'name': 'John',
    "surname": 'Smith',
    "company": 'Redis',
    "age": 29
})

0

In [19]:
r.hgetall('user-session:123')

{'name': 'John', 'surname': 'Smith', 'company': 'Redis', 'age': '29'}